<a href="https://colab.research.google.com/github/ykitaguchi77/YouTube_DL/blob/main/Faster_Whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U yt-dlp
!pip install -U faster_whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 99.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 114.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 120.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 13.2 MB/s eta 0:00:00
  Created wheel for faster_whisper: filename=faster_whisper-0.10.0-py3-no

In [ ]:
import subprocess
import re
from faster_whisper import WhisperModel

YOUTUBE_ID = "TOVAc1YCgag"  # Youtube ID
AUDIO_FILE_NAME = f"{YOUTUBE_ID}.mp3"

# Download audio and metadata from Youtube
def dl_yt(yt_url):
    # Download audio
    subprocess.run(f"yt-dlp -x --audio-format mp3 -o {AUDIO_FILE_NAME} {yt_url}", shell=True)
    # Get metadata
    result = subprocess.run(f"yt-dlp -j {yt_url}", shell=True, capture_output=True, text=True)
    return result.stdout

metadata = dl_yt(f"https://youtu.be/{YOUTUBE_ID}")

# Extract title from metadata
import json
metadata_json = json.loads(metadata)
title = metadata_json.get("title", YOUTUBE_ID)

# Sanitize the title to be used as a filename
safe_title = re.sub(r'[\\/*?:"<>|]', "", title)

TRANSCRIPTION_FILE_NAME = f"{safe_title}_transcription.txt"

model_size = "large-v2"
model = WhisperModel(model_size, device="cuda", compute_type="float16")
segments, info = model.transcribe(AUDIO_FILE_NAME, beam_size=5)

# Print detected language
print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

# Write the transcription to a file
with open(TRANSCRIPTION_FILE_NAME, "w", encoding="utf-8") as file:
    for segment in segments:
        file.write(f"{segment.text}\n")
        print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

print(f"Transcription saved to {TRANSCRIPTION_FILE_NAME}")


#**動画ID --> CHHANNEL_URL --> すべての動画をDL**

In [5]:
import subprocess
import re
import json
import os
from faster_whisper import WhisperModel

VIDEO_ID = "YaK2pwtavi4"  # 動画IDをここに設定
AUDIO_DIR = "downloaded_audio"
TRANSCRIPTION_DIR = "transcriptions"

# ディレクトリ確認・作成
os.makedirs(AUDIO_DIR, exist_ok=True)
os.makedirs(TRANSCRIPTION_DIR, exist_ok=True)

# YouTube動画からチャンネルURLを取得
def get_channel_url_from_video(video_id):
    result = subprocess.run(f"yt-dlp -j https://youtu.be/{video_id}", shell=True, capture_output=True, text=True)
    video_data = json.loads(result.stdout)
    return video_data["channel_url"]

def list_videos(channel_url):
    result = subprocess.run(f"yt-dlp -j --flat-playlist {channel_url}", shell=True, capture_output=True, text=True)
    lines = result.stdout.split('\n')  # 出力を行に分割
    videos = []
    for line in lines:
        if line:  # 空の行を無視
            try:
                video = json.loads(line)
                videos.append((video['id'], video['title']))
            except json.JSONDecodeError as e:
                print(f"JSON decoding error: {e} for line: {line}")
    return videos


# メイン処理
CHANNEL_URL = get_channel_url_from_video(VIDEO_ID)
videos = list_videos(CHANNEL_URL)

# 以下、既存のdl_yt, transcribe_and_save 関数と動画処理ループを使用
print(CHANNEL_URL)

https://www.youtube.com/channel/UCH9YK4B72yMjRPsVgx-EXxw


In [8]:
import subprocess
import re
import json
import os
from faster_whisper import WhisperModel

CHANNEL_URL = CHANNEL_URL
AUDIO_DIR = "downloaded_audio"
TRANSCRIPTION_DIR = "transcriptions"

# Ensure directories exist
os.makedirs(AUDIO_DIR, exist_ok=True)
os.makedirs(TRANSCRIPTION_DIR, exist_ok=True)

# Function to download audio and metadata from Youtube
def dl_yt(yt_url, audio_file_name):
    subprocess.run(f"yt-dlp -x --audio-format mp3 -o {AUDIO_DIR}/{audio_file_name} {yt_url}", shell=True)
    result = subprocess.run(f"yt-dlp -j {yt_url}", shell=True, capture_output=True, text=True)
    return result.stdout

# Function to list all videos from a channel
def list_videos(channel_url):
    result = subprocess.run(f"yt-dlp -j --flat-playlist {channel_url}", shell=True, capture_output=True, text=True)
    lines = result.stdout.strip().split('\n')  # Split the output into lines
    videos = []
    for line in lines:
        if line:  # Ignore empty lines
            try:
                video_data = json.loads(line)
                videos.append((video_data['id'], video_data['title']))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON for line: {line}\nError: {e}")
    return videos


# Function to transcribe and save audio
def transcribe_and_save(audio_file_name, transcription_file_name):
    model_size = "large-v2"
    model = WhisperModel(model_size, device="cuda", compute_type="float16")
    segments, info = model.transcribe(f"{AUDIO_DIR}/{audio_file_name}", beam_size=5)

    with open(f"{TRANSCRIPTION_DIR}/{transcription_file_name}", "w", encoding="utf-8") as file:
        for segment in segments:
            file.write(f"{segment.text}\n")
            print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))


# Fetch all videos from the channel
videos = list_videos(CHANNEL_URL)

# Process each video
for video_id, title in videos:
    yt_url = f"https://youtu.be/{video_id}"
    audio_file_name = f"{video_id}.mp3"
    safe_title = re.sub(r'[\\/*?:"<>|]', "", title)
    transcription_file_name = f"{safe_title}_transcription.txt"

    metadata = dl_yt(yt_url, audio_file_name)
    transcribe_and_save(audio_file_name, transcription_file_name)
    print(f"Processed video: {title}")


[0.00s -> 4.80s] モテない男は知らない女の子が好きになるきっかけ3選について話していきたいとおもいます
[4.80s -> 5.40s] どうもヨシです
[5.40s -> 7.60s] 皆さんですねこんなことになってませんかと
[7.60s -> 10.70s] 初対面で強い男をアピってもうまくいかないですと
[10.70s -> 13.00s] おらついたら怖いと言われましたとか
[13.00s -> 14.70s] 皆さんねこういう経験あると思います
[14.70s -> 17.10s] 致命的すぎる間違いをしているので
[17.10s -> 19.40s] 今回はそれを解説していきたいと思います
[19.40s -> 20.50s] 動画を見るメリットですね
[20.50s -> 26.10s] 女の子が好きになる過程と人の感情の動かし方の基礎が分かるようになっております
[26.10s -> 29.20s] なんで人の感情を動かさなあかんのって話もしてるんで
[29.20s -> 31.90s] ぜひですね今回の内容を覚えていってください
[31.90s -> 35.60s] 恋愛をなめくさってる男がまだ世の中には多いです
[35.60s -> 37.90s] そういうねバカどもを皆さん立ち抜いて
[37.90s -> 40.40s] さっさといい女の子とイチャイチャしましょう
[40.40s -> 42.70s] 今回の動画の内容を知らないデメリットですね
[42.70s -> 47.00s] 相手に応じての需要がわからずあっさり情報を拾っては捨てる
[47.00s -> 49.90s] 何も自分の能力がつかない60歳定年と
[49.90s -> 51.70s] はい言いすぎですけども
[51.70s -> 53.40s] 具体的なフレーズとかっていうのは
[53.40s -> 55.90s] 意味ないぜって言ってるのはこれのせいです
[55.90s -> 57.70s] 拾ってきては使って
[57.70s -> 59.60s] あー意味ねーやって捨てて
[59.60s -> 62.80s] 永遠ってやってると本当に何の実力もつかないで
[62.80s -> 64.30s] コミュニケーションも全くうまくない
[64.30s -> 66.

KeyboardInterrupt: ignored

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import subprocess
import os
import re
import json

VIDEO_ID = "YaK2pwtavi4"  # 動画IDを設定
BASE_PATH = "/content/drive/MyDrive/YouTube"  # ベースとなるパス

def get_channel_info_from_video(video_id):
    result = subprocess.run(f"yt-dlp -j https://youtu.be/{video_id}", shell=True, capture_output=True, text=True)
    video_data = json.loads(result.stdout)
    channel_url = video_data["channel_url"]
    channel_name = video_data["channel"]
    return channel_url, channel_name

def create_directories(channel_name):
    path = os.path.join(BASE_PATH, channel_name)
    os.makedirs(path, exist_ok=True)
    os.makedirs(os.path.join(path, "audio"), exist_ok=True)
    os.makedirs(os.path.join(path, "transcriptions"), exist_ok=True)
    return path

def list_videos(channel_url):
    result = subprocess.run(f"yt-dlp -j --flat-playlist {channel_url}", shell=True, capture_output=True, text=True)
    lines = result.stdout.strip().split('\n')
    videos = []
    for line in lines:
        if line:
            try:
                video_data = json.loads(line)
                videos.append((video_data['id'], video_data['title']))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON for line: {line}\nError: {e}")
    return videos

CHANNEL_URL, channel_name = get_channel_info_from_video(VIDEO_ID)
channel_path = create_directories(channel_name)
videos = list_videos(CHANNEL_URL)

print(channel_name)
#print(list_videos(CHANNEL_URL))

In [ ]:
# yt-dlpを使用して動画をダウンロード（例示）
def download_video(video_id, title, channel_path):
    safe_title = re.sub(r'[\\/*?:"<>|]', "", title)
    audio_file_path = os.path.join(channel_path, "audio", f"{safe_title}.mp3")
    subprocess.run(f"yt-dlp -x --audio-format mp3 -o {audio_file_path} https://youtu.be/{video_id}", shell=True)

# 以下の部分を動画リスト取得とダウンロードのループに組み込む
for video_id, title in videos:
    download_video(video_id, title, channel_path)


In [ ]:
from faster_whisper import WhisperModel
import os

def transcribe_and_save(audio_file_path, transcription_file_path):
    model_size = "large-v2"
    model = WhisperModel(model_size, device="cuda", compute_type="float16")
    segments, info = model.transcribe(audio_file_path, beam_size=5)

    with open(transcription_file_path, "w", encoding="utf-8") as file:
        for segment in segments:
            file.write(f"{segment.text}\n")

# 以下の部分をオーディオファイルのリスト処理ループに組み込む
transcribe_and_save(audio_file_path, transcription_file_path)
